# Frequent Pattern Study
## Litratures 

|Name|Source|
|:-|:-|
|Frequent Pattern Mining Charu C. Aggarwal, Jiawei Han | [![](../Litrature_Reviews/freq.PNG)]() |
|Frequent Itemsets via Apriori Algorithm|[Github User Guide for mlxtend](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)|

## Frequent Pattern using Apriori Algorithm

### Apriori function : 
- is an algorithm for frequent item set mining and association rule learning over relational databases.
- It proceeds by identifying the frequent individual items in the database and extending them to larger and larger item sets as long as those item sets appear sufficiently often in the database.




# Example 1 -- Generating Frequent Itemsets

The apriori function expects data in a `one-hot` encoded pandas DataFrame. Suppose we have the following transaction data:


In [1]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

- We can transform it into the right format via the TransactionEncoder as follows:

In [37]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder


In [3]:
te = TransactionEncoder()
# The fit method is to learn what is the item to transaction
# the transform method is to Transform transactions into a one-hot encoded NumPy array.
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


- Imagine that we need the min-support for transactions that fit 60%
- apriori function Get frequent itemsets from a one-hot DataFrame


In [4]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6 , use_colnames=True)

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Eggs, Onion)"
7,0.6,"(Kidney Beans, Milk)"
8,0.6,"(Kidney Beans, Onion)"
9,0.6,"(Kidney Beans, Yogurt)"


-  that means `Eggs` is frequently 80% repeated in all transactions and `Beans` is the Most frequent item with 100% 

# Association Rules Generation from Frequent Itemsets

`Rule generation` is a common task in the mining of frequent patterns.
-  An association rule is an implication expression of the form X → Y, where X and Y are disjoint itemsets

A more concrete example based on consumer behaviour would be `{Diapers}→{Beer}` suggesting that people who buy diapers are also likely to buy beer. 
- To evaluate the "interest" of such an association rule, different `metrics` have been developed. The current implementation make use of the `confidence` and `lift` metrics.

'support':
$support(A→C)=support(A∪C),range: [0,1]$

The `support` metric is defined for itemsets, not assocication rules. 
- The table produced by the association rule mining algorithm contains three different support metrics: 
    - 'antecedent support'
    - 'consequent support'
    - 'support'

> 'antecedent support' computes the proportion of transactions that contain the antecedent A.


> 'consequent support' computes the support for the itemset of the consequent C.


> The 'support' metric then computes the support of the combined itemset A ∪ C -- 
    -note that 'support' depends on 'antecedent support' and 'consequent support' via $$min('antecedent support', 'consequent support')$$ 

Typically, support is used to measure the `abundance` or `frequency` (often interpreted as significance or importance) of an itemset in a database. 

- We refer to an itemset as a __frequent itemset__ if you support is larger than a specified minimum-support threshold. 
    - Note that in general, due to the downward closure property, all subsets of a frequent itemset are also frequent.
    - here comes the pruning idea for apriori, Open lecture to understand more




'__confidence__': Let's create the Law with $LaTex$

$$ confidence(A→C) = {{support(A→C)} \over {support(A)}},range: [0,1]$$

> The confidence of a rule A->C is the probability of seeing the consequent in a transaction given that it also contains the antecedent. 

- Note that the metric is not symmetric or directed; for instance, the confidence for A->C is different than the confidence for C->A. 
- The confidence is 1 (maximal) for a rule A->C if the consequent and antecedent always occur together.

'lift':
$$lift(A→C)={{confidence(A→C)} \over {support(C)}},range: [0,∞]$$

- The lift metric is commonly used to measure how much more often the antecedent and consequent of a rule A->C occur together than we would expect if they were statistically independent. 
- If A and C are independent, the Lift score will be exactly 1.

In [36]:
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(df, min_support=0.6 , use_colnames=True)

associationRulesSet =association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
associationRulesSet

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.0
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf
2,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
3,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
4,(Milk),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
5,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
6,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
7,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
8,"(Kidney Beans, Onion)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
9,"(Eggs, Onion)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf


- I Wouldn't like to manipulate the frozen sets so we make a list
- The Association Rules can be defined as : $$antecedent -> consequent = [ support = # , confidence = #]$$

In [22]:
for i , row in associationRulesSet.iterrows():
    print(f"{i+1} - {list(row.antecedents)} -> {list(row.consequents)} = [support = {row.support} , confidence = {row.confidence}, lift ={row.lift}] , dependent = {row.lift != 1.0}" )




1 - ['Kidney Beans'] -> ['Eggs'] = [support = 0.8 , confidence = 0.8, lift =1.0] , dependent = False
2 - ['Eggs'] -> ['Kidney Beans'] = [support = 0.8 , confidence = 1.0, lift =1.0] , dependent = False
3 - ['Eggs'] -> ['Onion'] = [support = 0.6 , confidence = 0.7499999999999999, lift =1.2499999999999998] , dependent = True
4 - ['Onion'] -> ['Eggs'] = [support = 0.6 , confidence = 1.0, lift =1.25] , dependent = True
5 - ['Milk'] -> ['Kidney Beans'] = [support = 0.6 , confidence = 1.0, lift =1.0] , dependent = False
6 - ['Onion'] -> ['Kidney Beans'] = [support = 0.6 , confidence = 1.0, lift =1.0] , dependent = False
7 - ['Yogurt'] -> ['Kidney Beans'] = [support = 0.6 , confidence = 1.0, lift =1.0] , dependent = False
8 - ['Kidney Beans', 'Eggs'] -> ['Onion'] = [support = 0.6 , confidence = 0.7499999999999999, lift =1.2499999999999998] , dependent = True
9 - ['Kidney Beans', 'Onion'] -> ['Eggs'] = [support = 0.6 , confidence = 1.0, lift =1.25] , dependent = True
10 - ['Eggs', 'Onion'] -> 

In [23]:
associationRulesSet["antecedent_len"] = associationRulesSet["antecedents"].apply(lambda x: len(x))
associationRulesSet



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.0,1
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf,1
2,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1
3,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,1
4,(Milk),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,1
5,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,1
6,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,1
7,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,2
8,"(Kidney Beans, Onion)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,2
9,"(Eggs, Onion)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,2


- if you just need the associaltion rule of some antecedents or consequents

In [26]:
associationRulesSet[associationRulesSet['antecedents'] == {'Eggs', 'Kidney Beans'}]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
7,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,2


In [27]:
associationRulesSet[associationRulesSet['consequents'] == {'Onion'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
2,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1
7,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,2


if you need to query something by rules just do it :

- rules 
    - support >= 0.7
    - confidence >= 0.9
    - lenth of items as anticedents = 1

In [33]:
associationRulesSet[(associationRulesSet['support'] >= 0.7) &
                    (associationRulesSet['confidence'] >=0.8)&
                    (associationRulesSet['antecedent_len'] ==1) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.8,1.0,0.0,1.0,1
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.0,1.0,0.0,inf,1


## Let's Going to the Lecture Example



In [39]:
Transactions=  [['l1', 'l2', 'l5'],
                ['l2', 'l4'],
                ['l2', 'l3'],
                ['l1', 'l2', 'l4'],
                ['l1', 'l3'],
                ['l2', 'l3'],
                ['l1', 'l3'],
                ['l1', 'l2', 'l3', 'l5'],
                ['l1', 'l2', 'l3'],
                ['l1', 'l2']]

In [40]:
te = TransactionEncoder()
transactions_ar = te.fit(Transactions).transform(Transactions)
TrDataframe = pd.DataFrame(transactions_ar ,  columns= te.columns_)
TrDataframe

,l1,l2,l3,l4,l5
0,True,True,False,False,True
1,False,True,False,True,False
2,False,True,True,False,False
3,True,True,False,True,False
4,True,False,True,False,False
5,False,True,True,False,False
6,True,False,True,False,False
7,True,True,True,False,True
8,True,True,True,False,False
9,True,True,False,False,False


- now we are using apriori with minimum support in our example is about 2 of 10 transaction so the min_support = 0.2

In [42]:
frequentSet =apriori(TrDataframe, min_support= 0.2 , use_colnames=True)
frequentSet

,support,itemsets
0,0.7,(l1)
1,0.8,(l2)
2,0.6,(l3)
3,0.2,(l4)
4,0.2,(l5)
5,0.5,"(l1, l2)"
6,0.4,"(l1, l3)"
7,0.2,"(l5, l1)"
8,0.4,"(l2, l3)"
9,0.2,"(l2, l4)"


-دلوقتى الحمد لله جبنا نتايج الشغل صحيح مقارنة بالمحاضرة 
- 
![](min_confSlide.PNG)

In [44]:
rules =association_rules(frequentSet, metric="confidence", min_threshold=0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(l1),(l2),0.7,0.8,0.5,0.714286,0.892857,-0.06,0.7
1,(l5),(l1),0.2,0.7,0.2,1.000000,1.428571,0.06,inf
2,(l4),(l2),0.2,0.8,0.2,1.000000,1.250000,0.04,inf
3,(l5),(l2),0.2,0.8,0.2,1.000000,1.250000,0.04,inf
4,"(l5, l1)",(l2),0.2,0.8,0.2,1.000000,1.250000,0.04,inf
5,"(l5, l2)",(l1),0.2,0.7,0.2,1.000000,1.428571,0.06,inf
6,(l5),"(l1, l2)",0.2,0.5,0.2,1.000000,2.000000,0.10,inf


# Frequent Itemsets via the FP-Growth Algorithm

In [6]:
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,1.0,(Kidney Beans)
1,0.8,(Eggs)
2,0.6,(Yogurt)
3,0.6,(Onion)
4,0.6,(Milk)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Yogurt, Kidney Beans)"
7,0.6,"(Eggs, Onion)"
8,0.6,"(Kidney Beans, Onion)"
9,0.6,"(Kidney Beans, Eggs, Onion)"


In [7]:
%timeit -n 100 -r 10 apriori(df, min_support=0.6)

7.55 ms ± 1.31 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [8]:
%timeit -n 100 -r 10 fpgrowth(df, min_support=0.6)

3.36 ms ± 681 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
